In [1]:
!pip install transformers
!pip install SentencePiece
!pip install datasets
!pip install pytorch_lightning


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.7

In [2]:
!pip install bertviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 94.1 MB/s eta 0:00:00


In [6]:
from google.colab import drive
import os
drive.mount('/content/drive')
DIR = '/content/drive/MyDrive/Colab Notebooks/w266/Final_Project'
os.chdir(DIR)

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt

import re
import itertools

import torch


from nltk.translate.gleu_score import corpus_gleu, sentence_gleu
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu


from transformers import T5Tokenizer, TFT5Model, T5ForConditionalGeneration
from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate


from torch.utils.data import Dataset, DataLoader
import datasets

from transformers import Adafactor, get_linear_schedule_with_warmup
import pytorch_lightning as pl

from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning import Trainer

import warnings
warnings.filterwarnings("ignore")


In [7]:
device = torch.device('cuda:0')

In [8]:
class quoteT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=3, warmup_steps=1000):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained("t5-small")
        self.save_hyperparameters()

        self.train_losses = []
        self.val_losses = []


    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids.squeeze(1), attention_mask=attention_mask.squeeze(1), labels=labels.squeeze(1))
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("training_loss", loss)
        self.train_losses.append(loss.item())

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)
        self.val_losses.append(loss.item())

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        optimizer = Adafactor(self.parameters(), relative_step=True, warmup_init=True, lr=None)     
        return {"optimizer": optimizer}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return validation_dataloader

    def test_dataloader(self):
        return test_dataloader

In [9]:
#importing the model from storage
trained = torch.load(DIR + '/model/model4-large5.pt')
trained.to(device)


quoteT5(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=1024, out_features=4096, bias=False)
                (wo): Linear(in_

In [19]:
generate_quote('Write a quote about love from the perspective of Jane Austen', 2, 3)

  0%|          | 0/1 [00:00<?, ?it/s]


('I love a man who can make me laugh, but not make me cry.',
 tensor(80.5970, device='cuda:0'))

In [10]:
#Running a forward pass to get attention scores
tokenizer = T5Tokenizer.from_pretrained('t5-large')
encoder_input_ids = tokenizer("Write a quote about love from the perspective of Dr. Seuss", return_tensors="pt", add_special_tokens=True).input_ids
with tokenizer.as_target_tokenizer():
    decoder_input_ids = tokenizer("You know you're in love when you can't fall asleep because reality is finally better than your dreams.", return_tensors="pt", add_special_tokens=True).input_ids

outputs = trained.model(input_ids=encoder_input_ids.to(device), decoder_input_ids=decoder_input_ids.to(device), output_attentions=True)

encoder_text = tokenizer.convert_ids_to_tokens(encoder_input_ids[0])
decoder_text = tokenizer.convert_ids_to_tokens(decoder_input_ids[0])

In [11]:
from bertviz import model_view
model_view(
    encoder_attention=outputs.encoder_attentions,
    decoder_attention=outputs.decoder_attentions,
    cross_attention=outputs.cross_attentions,
    encoder_tokens= encoder_text,
    decoder_tokens = decoder_text
)

# Evaluation and testing of t5-large

---

Sample 400 examples from the dataset, generate quotes, and evaluate BLEU, GLUE, and Perplexity.


In [ ]:
df = pd.read_csv('final_quotes.csv')
df = df[~df.quote.isna()]
df = df[df['quote'].str.split().apply(len) <= 50]
df['inputs'] = df.apply(lambda x: "Write a quote about {} from the perspective of {}".format(x['tags'], x['auth']), axis=1)

In [ ]:
df = df[~df.tags.isna()]

In [ ]:
test = df.sample(400)


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-large')

In [ ]:
trained = torch.load(DIR + '/model/model4-large5.pt')
trained.to(device)

quoteT5(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=1024, out_features=4096, bias=False)
                (wo): Linear(in_

In [8]:
from tqdm import tqdm

def generate_quote(request, beam=4, ngram=3):

    inputs_encoding =  tokenizer(
        request,
        add_special_tokens=True,
        max_length= 15,
        padding = 'max_length',
        truncation='only_first',
        return_attention_mask=True,
        return_tensors="pt"
        )

    
    generate_ids = trained.model.generate(
        input_ids = inputs_encoding["input_ids"].to(device),
        attention_mask = inputs_encoding["attention_mask"].to(device),
        max_length = 50,
        num_beams = beam,
        num_return_sequences = 1,
        no_repeat_ngram_size=ngram,
        early_stopping=True,
        )

    preds = [
        tokenizer.decode(gen_id,
        skip_special_tokens=True, 
        clean_up_tokenization_spaces=True)
        for gen_id in generate_ids
    ]


    max_length = 50
    stride = 512
    seq_len = inputs_encoding.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = inputs_encoding.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = trained.model(input_ids.to(device), labels=target_ids.to(device), attention_mask = inputs_encoding.attention_mask.to(device))
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
     
    return ("".join(preds), ppl)

In [ ]:
from nltk.translate.bleu_score import SmoothingFunction
beam_widths = [2,3,4]
n_grams = [2,3,4]
smooth_fn = SmoothingFunction().method1
k=0.1
results_no_tune = {}

for beam_width, n_gram in itertools.product(beam_widths, n_grams):
  perplexities = []
  bleu = []
  gleu = []
  for idx, row in test.iterrows():    
    result, perplexity = generate_quote(row['inputs'], beam=beam_width, ngram=n_gram)
    perplexities.append(float(perplexity))
    if idx % 25 == 0:
      torch.cuda.empty_cache()
    test_set = df[(df['tags'] == row['tags']) & (df['auth'] == row['auth'])].quote
    
    reference_quotes = [quote.split() for quote in test_set.values]
    generated_quotes = result.split()

    bleu.append(sentence_bleu(reference_quotes, generated_quotes, smoothing_function=smooth_fn, weights=(0.25, 0.25, 0.25, 0.25), auto_reweigh=True))
    gleu.append(sentence_gleu(reference_quotes, generated_quotes))

    # store the results in the dictionary

  bleu = np.array(bleu)
  gleu = np.array(gleu)
  perplexity = np.array(perplexities)

  results_no_tune[str((beam_width, n_gram))] = {'GLEU': gleu.mean(), 'BLEU': bleu.mean(), 'PERPLEXITY': perplexity.mean()}
    
# analyze the results to determine the best combination of beam_width and n-gram
best_combination_gleu = max(results_no_tune, key=lambda x: results_no_tune[x]['GLEU'])
best_combination_bleu = max(results_no_tune, key=lambda x: results_no_tune[x]['BLEU'])
best_combination_perp = min(results_no_tune, key=lambda x: results_no_tune[x]['PERPLEXITY'])

  0%|          | 0/1 [00:00<?, ?it/s]


In [ ]:
df = pd.DataFrame(results_no_tune).transpose().round(4)

In [ ]:
df

,GLEU,BLEU,PERPLEXITY
"(2, 2)",0.1239,0.1036,286.2031
"(2, 3)",0.1356,0.1169,285.3700
"(2, 4)",0.1338,0.1172,286.4301
"(3, 2)",0.1294,0.1092,284.3644
"(3, 3)",0.1273,0.1054,285.8423
"(3, 4)",0.1357,0.1158,294.9742
"(4, 2)",0.1231,0.1018,276.9783
"(4, 3)",0.1334,0.1138,295.0190
"(4, 4)",0.1400,0.1198,287.2585


# T5 - Base Eval

In [20]:
torch.cuda.clear_cache()

AttributeError: ignored

In [21]:
trained = torch.load(DIR + '/model/model4-base-2final.pt')
trained.to(device)


quoteT5(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_features=30

In [28]:
generate_quote('Write a quote about love from the perspective of Jane Austen', 2, 3)

  0%|          | 0/1 [00:00<?, ?it/s]


('It is not fair to compare the love of someone with whom nothing could be more disagreeable.',
 tensor(249.4002, device='cuda:0'))

In [ ]:
from tqdm import tqdm

def generate_quote(request, beam=4, ngram=3):

    inputs_encoding =  tokenizer(
        request,
        add_special_tokens=True,
        max_length= 15,
        padding = 'max_length',
        truncation='only_first',
        return_attention_mask=True,
        return_tensors="pt"
        )

    
    generate_ids = trained.model.generate(
        input_ids = inputs_encoding["input_ids"].to(device),
        attention_mask = inputs_encoding["attention_mask"].to(device),
        max_length = 50,
        num_beams = beam,
        num_return_sequences = 1,
        no_repeat_ngram_size=ngram,
        early_stopping=True,
        )

    preds = [
        tokenizer.decode(gen_id,
        skip_special_tokens=True, 
        clean_up_tokenization_spaces=True)
        for gen_id in generate_ids
    ]


    max_length = 50
    stride = 512
    seq_len = inputs_encoding.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = inputs_encoding.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = trained.model(input_ids.to(device), labels=target_ids.to(device), attention_mask = inputs_encoding.attention_mask.to(device))
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
     
    return ("".join(preds), ppl)

In [ ]:
from nltk.translate.bleu_score import SmoothingFunction
beam_widths = [2,3,4]
n_grams = [2,3,4]
smooth_fn = SmoothingFunction().method1
k=0.1
results_no_tune = {}

for beam_width, n_gram in itertools.product(beam_widths, n_grams):
  perplexities = []
  bleu = []
  gleu = []
  for idx, row in test.iterrows():    
    result, perplexity = generate_quote(row['inputs'], beam=beam_width, ngram=n_gram)
    perplexities.append(float(perplexity))
    if idx % 25 == 0:
      torch.cuda.empty_cache()
    test_set = df[(df['tags'] == row['tags']) & (df['auth'] == row['auth'])].quote
    
    reference_quotes = [quote.split() for quote in test_set.values]
    generated_quotes = result.split()

    bleu.append(sentence_bleu(reference_quotes, generated_quotes, smoothing_function=smooth_fn, weights=(0.25, 0.25, 0.25, 0.25), auto_reweigh=True))
    gleu.append(sentence_gleu(reference_quotes, generated_quotes))

    # store the results in the dictionary

  bleu = np.array(bleu)
  gleu = np.array(gleu)
  perplexity = np.array(perplexities)

  results_no_tune[str((beam_width, n_gram))] = {'GLEU': gleu.mean(), 'BLEU': bleu.mean(), 'PERPLEXITY': perplexity.mean()}
    
# analyze the results to determine the best combination of beam_width and n-gram
best_combination_gleu = max(results_no_tune, key=lambda x: results_no_tune[x]['GLEU'])
best_combination_bleu = max(results_no_tune, key=lambda x: results_no_tune[x]['BLEU'])
best_combination_perp = min(results_no_tune, key=lambda x: results_no_tune[x]['PERPLEXITY'])

  0%|          | 0/1 [00:00<?, ?it/s]


In [ ]:
df = pd.DataFrame(results_no_tune).transpose().round(4)

In [ ]:
df

,GLEU,BLEU,PERPLEXITY
"(2, 2)",0.1417,0.1231,2231.2104
"(2, 3)",0.1392,0.1205,1794.5572
"(2, 4)",0.1413,0.1236,2100.6114
"(3, 2)",0.1464,0.1295,2351.4219
"(3, 3)",0.1552,0.1361,1693.1771
"(3, 4)",0.1392,0.1194,1824.3087
"(4, 2)",0.1463,0.1296,1749.2210
"(4, 3)",0.1489,0.1301,2554.4302
"(4, 4)",0.1480,0.1303,1847.5108
